# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [105]:
import requests
import time
import re
import typing

from functools import wraps
from random import randint

In [86]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [87]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """

    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'Время выполнения функции {func.__name__}: {end - start}')
        return result

    return wrapper

## Задание 2

In [88]:
def logging(func):
    """
    Декоратор, который выводит параметры с которыми была вызвана функция
    """

    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print('Функция вызвана с параметрами:')
        print(f'{args}, {kwargs}')
        return result

    return wrapper

## Задание 3

In [89]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """

    call_count = 0

    @wraps(func)
    def wrapper(*args, **kwargs):
        nonlocal call_count
        result = func(*args, **kwargs)
        call_count += 1
        print(f'Функция была вызвана: {call_count} раз')
        return result

    wrapper.call_count = call_count

    return wrapper

## Задание 4

In [106]:
def memo(func):
    """
    Декоратор, запоминающий результаты исполнения функции func, чьи аргументы args должны быть хешируемыми
    """
    cache = {}

    @wraps(func)
    def fmemo(*args):
        if all(isinstance(arg, typing.Hashable) for arg in args):
            if args in cache:
                return cache[args]
            else:
                result = func(*args)
                cache[args] = result
                return result
        else:
            return func(*args)

    fmemo.cache = cache
    return fmemo

## Тестирование

In [104]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))
print(word_count('whole'))

Время выполнения функции word_count: 0.5191234019998774
Функция вызвана с параметрами:
('whole',), {}
Функция была вызвана: 1 раз
Cлово whole встречается 176 раз
Время выполнения функции word_count: 0.5850093740000375
Функция вызвана с параметрами:
('whole',), {}
Функция была вызвана: 2 раз
Cлово whole встречается 176 раз


In [98]:
F_COUNT = 30

In [99]:
@benchmark
def fib_benchmark(func):
    print(func(F_COUNT))

In [100]:
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [101]:
# измеряем время выполнения
fib_benchmark(fib)

832040
Время выполнения функции fib_benchmark: 0.3507692069999848


In [107]:
@memo
def fib_memo(n):
    if n < 2:
        return n
    return fib_memo(n-2) + fib_memo(n-1)

In [110]:
# измеряем время выполнения
fib_benchmark(fib_memo)

832040
Время выполнения функции fib_benchmark: 0.000915259000066726
